<div class="markdown-google-sans">

<a name="machine-learning-examples"></a>

### Featured examples

</div>

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
file_path = 'hotel_with_restaurantf.csv'
df = pd.read_csv(file_path)

# Fill missing values with 2 for rating columns and ensure all values are numeric
rating_columns = ['Service Quality', 'Amenities', 'Food and Drinks', 'Value for Money', 'Location', 'Cleanliness', 'multi_cuisine_restaurant']
df[rating_columns] = df[rating_columns].apply(pd.to_numeric, errors='coerce').fillna(2)

# Calculate composite score
df['Score'] = df[rating_columns].mean(axis=1)

# Define features and target
X = df[rating_columns]
y = df['Score']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

def get_property_types(city, df):
    # Get unique property types in the selected city
    property_types = df[df['city'] == city]['property_type'].unique()
    return property_types

def recommend_hotels(city, room_count, property_type, df, model):
    # Filter by city
    city_df = df[df['city'] == city]

    # Filter by room count
    city_df = city_df[city_df['room_count'] >= room_count]

    # If property type is specified, filter by property type
    if property_type:
        city_df = city_df[city_df['property_type'] == property_type]

    # If no hotels match the filters, return the highest score hotels in the city
    if city_df.empty:
        city_df = df[df['city'] == city]

    # Predict scores for filtered hotels
    features = city_df[rating_columns]
    city_df['Predicted_Score'] = model.predict(features)

    # Get the top 5 hotels with the highest predicted score
    top_5_hotels = city_df.nlargest(5, 'Predicted_Score')

    # Select only the required columns
    top_5_hotels = top_5_hotels[['latitude', 'longitude', 'property_name', 'address', 'city']]

    return top_5_hotels

# Example usage
city = input("Enter city: ")
property_types = get_property_types(city, df)
print(f"Available property types in {city}: {property_types}")

property_type = input("Select property type from above (or press Enter to skip): ")
room_count = int(input("Enter required number of rooms: "))

top_5_hotels = recommend_hotels(city, room_count, property_type if property_type else None, df, model)
print(f"The top 5 hotels are:\n{top_5_hotels}")

Mean Squared Error: 0.0007846803316326537
Enter city: Manali
Available property types in Manali: ['Resort' 'Cottage' 'Villa' 'Hotel' 'Tent' 'BnB' 'Farm Stay' 'Homestay']
Select property type from above (or press Enter to skip): Resort
Enter required number of rooms: 2
The top 5 hotels are:
       latitude  longitude                       property_name  \
1590  32.206290  77.190559               Citrus Manali Resorts   
2659  32.207688  77.200419       The Serenity Resort &amp; Spa   
2869  32.222023  77.189235    Thomas Villa, Hotel and Cottages   
3169  32.219267  77.184638        Hotel Royal Orchard - Manali   
473   32.137297  77.170665  Bharhka Countryside Cottage Resort   

                                                address    city  
1590  Kullu Manali National Highway, Manali, Distric...  Manali  
2659  Near Shaminala, Village Jagatsukh, Naggar High...  Manali  
2869       Kanyal Road, Manali, Himachal Pradesh (H.P.)  Manali  
3169                      Kanyal road, Gadherni,